chroma DB

In [12]:
import os
os.environ['OPENAI_API_KEY'] = "sk"

In [24]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader

In [25]:
loader = PyPDFDirectoryLoader("pdfs")

In [27]:
document = loader.load()

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(document)

In [29]:
len(text)

16

In [32]:
text[15].page_content

'transitions (race conditions) and ensures reliable sequential operation in digital circuits. \n \nThe most critical timing parameters are: \n- Setup time: D must be stable before the clock edge \n- Hold time: D must remain stable after the clock edge \n- Clock-to-Q delay: Time taken for the output to reflect the captured input \n \nThis implementation using NAND gates and the master-slave architecture provides good \nnoise immunity and reliable operation, making it suitable for use in sequential digital \ncircuits, registers, and other synchronous applications.'

Creating DB

In [ ]:
from langchain import embeddings

In [ ]:
persist_directory = 'db'

In [ ]:
embedding = OpenAIEmbeddings()

In [ ]:
vectordb = Chroma.from_documents(documents=text,embedding=embedding,persist_directory=persist_directory)

In [ ]:
vectordb.persist()

In [ ]:
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,embedding_function=embedding)

In [ ]:
vectordb

make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")

In [ ]:
print(docs[0].page_content)

In [ ]:
len(docs)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.search_kwargs

In [ ]:
docs2 = retriever.get_relevant_documents("How much money did Microsoft raise?")

In [ ]:
len(docs2)

In [ ]:
docs2

make a chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
llm=OpenAI()

In [ ]:
llm

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Microsoft raise?"

In [ ]:
llm_response = qa_chain(query)

In [ ]:
llm_response

In [ ]:
process_llm_response(llm_response)

delete the db

In [ ]:
!zip -r db.zip ./db

In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

In [ ]:
# delete the directory
!rm -rf db/